# This code plots the classical confidence interval using Neyman's construction. i.e. for every value of mu (signal events), it gets the UL on n (number of data events) using poisson statistics

BKg is taken to be 3 for this case. We try to reproduce arXiv: 9711021 (Feldman's and Cousin's paper) fig 6 

In [17]:
###Developer: Shilpi Jain

import numpy as np
from scipy.stats import norm
import ROOT
from scipy.stats import poisson
from array import array
import math

In [18]:
## Generate a few values of mu from 0 to 20 with spacing 21./500. One can increase the spacing as much as one wants or put it to be 1
#mu_true = np.linspace(0, 20, 20001)
mu_true = np.linspace(0, 20, 500)
print(mu_true)
len(mu_true)
bkg = 3

[ 0.          0.04008016  0.08016032  0.12024048  0.16032064  0.2004008
  0.24048096  0.28056112  0.32064128  0.36072144  0.4008016   0.44088176
  0.48096192  0.52104208  0.56112224  0.6012024   0.64128257  0.68136273
  0.72144289  0.76152305  0.80160321  0.84168337  0.88176353  0.92184369
  0.96192385  1.00200401  1.04208417  1.08216433  1.12224449  1.16232465
  1.20240481  1.24248497  1.28256513  1.32264529  1.36272545  1.40280561
  1.44288577  1.48296593  1.52304609  1.56312625  1.60320641  1.64328657
  1.68336673  1.72344689  1.76352705  1.80360721  1.84368737  1.88376754
  1.9238477   1.96392786  2.00400802  2.04408818  2.08416834  2.1242485
  2.16432866  2.20440882  2.24448898  2.28456914  2.3246493   2.36472946
  2.40480962  2.44488978  2.48496994  2.5250501   2.56513026  2.60521042
  2.64529058  2.68537074  2.7254509   2.76553106  2.80561122  2.84569138
  2.88577154  2.9258517   2.96593186  3.00601202  3.04609218  3.08617234
  3.12625251  3.16633267  3.20641283  3.24649299  3.2

In [19]:
def get_lower_limit_pois(mu,b, prob):
    tmp_prob = 0
    n = 0
    nbest = mu+b
    while ( (tmp_prob <= prob) and abs(tmp_prob-prob)>0.01 ):
        tmp_prob += math.exp(-nbest) * math.pow(nbest,n)/math.factorial(n)
        #print(f"{n}:{tmp_prob}")
        n = n+1    
        
    #print(f"mu : n : tmp_prob {mu} {n-1} {tmp_prob}")          
    return n-1

def get_upper_limit_pois(mu,b, prob):
    tmp_prob = 0
    n = 0
    nbest = mu+b
    #while ( 1-tmp_prob >= prob ):
    while ( (1-tmp_prob >= prob) and abs(1-tmp_prob-prob)>0.01  ):
        tmp_prob = tmp_prob + math.exp(-nbest) * math.pow(nbest,n)/math.factorial(n)
        #print(f"{n}:{1-tmp_prob}")
        n = n+1
    #print(f"mu : n : tmp_prob {mu} {n-1} {1-tmp_prob}") 
    return n-1

In [20]:
Lower_limit = []
Upper_limit = []
muArr = []
np.arange(0,10)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [21]:
ntot = 0

### now loop over mu and the upper and lower limits on n

for mu in mu_true:
        #mu = 0.1
        #print(f"mu is {mu}")
        beta = 0.05
        alpha = 0.05
        t_beta = get_lower_limit_pois(mu,bkg,beta)
        t_alpha = get_upper_limit_pois(mu,bkg,alpha)
        
        Lower_limit.append(t_beta)
        Upper_limit.append(t_alpha)
        muArr.append(mu)
        ntot += 1
        #print(f"mu - t_beta + t_alpha {mu} {t_beta} : {t_alpha}")
        #############################################################

In [22]:
'''
step_upper = np.where(np.diff((np.array(Upper_limit))) != 0)
step_lower = np.where(np.diff((np.array(Lower_limit))) != 0)

mu_lo = mu_true[step_lower]
mu_lo = array("d", mu_lo)

mu_up = mu_true[step_upper]
mu_up = array("d", mu_up)


bin_center1 = np.arange(0.5, 16.5, 1.0)
bin_center1 = array("d", bin_center1)


bin_center2 = np.arange(5.5, 31.5, 1.0)
bin_center2 = array("d", bin_center2)

hlower_limit = ROOT.TGraph(len(mu_lo), bin_center1, mu_lo)
hupper_limit = ROOT.TGraph(len(mu_up), bin_center2, mu_up)
'''

'''
Lower_limit = array("d", Lower_limit)
Upper_limit = array("d", Upper_limit)
muArr = array("d", muArr)
hlower_limit = ROOT.TGraph(ntot, muArr,np.array(Lower_limit) )
hupper_limit = ROOT.TGraph(ntot, muArr, np.array(Upper_limit))
'''


### switch the axis to get the exact plot as in Feldman's and Cousin's paper in fig 6
Lower_limit_tmp = Upper_limit
Upper_limit_tmp = Lower_limit
Lower_limit = array("d", Lower_limit_tmp)
Upper_limit = array("d", Upper_limit_tmp)
muArr = array("d", muArr)
hupper_limit = ROOT.TGraph(ntot, np.array(Upper_limit), muArr)
hlower_limit = ROOT.TGraph(ntot, np.array(Lower_limit), muArr )





In [23]:
# Create TMultiGraph
TG = ROOT.TMultiGraph()
hupper_limit.SetMarkerStyle(20)  # 20 corresponds to a filled circle
hlower_limit.SetMarkerStyle(20)
hupper_limit.SetMarkerColor(2)
hlower_limit.SetMarkerColor(4)

TG.Add(hlower_limit, "AP")
TG.Add(hupper_limit, "AP")


# Create canvas
canvas = ROOT.TCanvas("canvas", "Confidence belt for Poisson distribution", 800, 600)
# Enable the grid for the canvas
canvas.SetGrid()
# Draw TMultiGraph
TG.Draw("A")

#TG.GetXaxis().SetTitle("Signal mean Mu")
#TG.GetYaxis().SetTitle("Measured n")

TG.GetXaxis().SetTitle("Measured n")
TG.GetYaxis().SetTitle("Signal mean Mu")

TG.GetXaxis().SetRangeUser(0, 15)
TG.GetYaxis().SetRangeUser(0, 15)
# Show the canvas
canvas.SaveAs("output.jpg")


Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: jpg file output.jpg has been created
